<a href="https://colab.research.google.com/github/smwongela/NLP_Transformers/blob/main/mt5_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers -q

     |████████████████████████████████| 249 kB 31.9 MB/s 
     |████████████████████████████████| 9.1 MB 55.6 MB/s 
     |████████████████████████████████| 1.8 MB 50.9 MB/s 
     |████████████████████████████████| 6.6 MB 56.4 MB/s 
     |████████████████████████████████| 362 kB 66.4 MB/s 
     |████████████████████████████████| 4.4 MB 68.4 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 70.3 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 181 kB 71.4 MB/s 
     |████████████████████████████████| 145 kB 73.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 140 kB 74.1 MB/s 
     |████████████████████████████████| 212 kB 68.0 MB/s 
     |████████████████████████████████| 1.1 MB 60.8 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
     |████████████

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

In [ ]:
import torch
torch.cuda.get_device_name(0)


'Tesla T4'

In [ ]:
import torch
import numpy as np
import os
import random

def set_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.enabled = False 
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(7)


In [ ]:
import wandb

In [ ]:
wandb.login(key="7a38c1f02cd649be5409f59df57bb45e746b4072")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!pwd

In [ ]:
import pandas as pd
path_to_dataset= "gdrive/My Drive/yoruba_english/"
train= pd.read_csv(os.path.join(path_to_dataset, "Train.csv"))
train.head()

In [ ]:
#Remove  any possible duplicate
train= train.drop_duplicates(subset=["Yoruba","English"])
#Lowercase and remove trailing spaces
train["Yoruba"] = train.apply(lambda x: (x.Yoruba).strip().lower(), axis=1)
train["English"]=train.English.apply(lambda x: x.lower())

train = train[["Yoruba","English"]]
train.columns=["input_text", "target_text"]

#Train 95%/ Validation 5% Split
validation = train.sample(frac=0.05).astype(str)
train=train.drop(index=validation.index).astype(str)

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_df = train
eval_df = validation

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 100
model_args.train_batch_size = 10
model_args.eval_batch_size = 10
model_args.num_train_epochs = 8
model_args.scheduler = "cosine_schedule_with_warmup"
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 10000
model_args.learning_rate = 0.0003
model_args.optimizer = 'Adafactor'
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.preprocess_inputs = False
model_args.use_early_stopping = True
model_args.num_return_sequences = 1
model_args.do_lower_case = True
model_args.output_dir = "all/"
model_args.best_model_dir = "all/best_model"
model_args.wandb_project = "Yoruba fine tune"

model = T5Model ("mt5", "google/mt5-base", args=model_args)
#model.model.load_state_dict(torch.load("../input/semifinalyoruba/outputs/best_model/pytorch_model.bin"))

# Train the model
model.train_model(train_df, eval_data=eval_df)
# Optional: Evaluate the model. We'll test it properly anyway.
results = model.eval_model(eval_df, verbose=True)


In [ ]:
model_args = T5Args()
model_args.max_length = 100
model_args.length_penalty = 2.5
model_args.repetition_penalty = 1.5
model_args.num_beams = 5

model = T5Model("mt5", "all/best_model", args=model_args)

In [ ]:
preds = model.predict(validation.input_text.values.tolist())

In [ ]:
validation["preds"] = preds